In [1]:
# load the library
import numpy as np
import pandas as pd
from tqdm import tqdm

# load selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pyperclip
from IPython.display import clear_output
import sqlite3
from pandas.io import sql
import datetime

In [2]:
# handling page refresh
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [3]:
# Set name of your SQLite database
db_name = "freight"

sqlite_db = db_name + '.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

In [4]:
# load the library
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd
from tqdm import tqdm
pd.set_option('max_colwidth',1000)
import time
from sklearn.externals import joblib

In [5]:
freight_lanes = pd.read_excel('freight_lanes.xlsx')

In [6]:
# download chromedriver.exe and save to a local drive locaiton:
location_of_chromedriver = "Driver/chromedriver.exe"
site_url = "https://www.bigschedules.com/routes"

In [7]:
# set browser and go to google maps
def init_browser(URL = 'https://www.google.com.hk/maps'):
    browser = webdriver.Chrome(location_of_chromedriver)
    browser.get(URL)
    return browser

In [128]:
# open table management

def get_tables(otm, row_no=6):
    html = otm.page_source
    data_df_day = pd.DataFrame()
    data_df_direct = pd.DataFrame()
    
    bs=BeautifulSoup(html, "lxml")
    rows = bs.find_all('tr')
    tbl_day = []
    tbl_direct = []
    tbl_result = []
    for row in rows:
        cols=row.find_all('td')
        colstrip=[x.text.strip() for x in cols]
    #     print(len(colstrip))
        tbl_result.append(colstrip)
    
#     get_lst = []
    for i in range(len(tbl_result)):
        if len(tbl_result[i]) == row_no:
            if "day" in tbl_result[i][0]:
                tbl_day.append(tbl_result[i])
                tbl_direct.append(tbl_result[i+1])
    
    df_tmp_day = pd.DataFrame(tbl_day)
    df_tmp_direct = pd.DataFrame(tbl_direct)
    
    data_df_day = data_df_day.append(df_tmp_day)
    data_df_direct = data_df_direct.append(df_tmp_direct)
    
    result = pd.concat([data_df_day, data_df_direct], axis=1, join='inner')
    
    return result

In [38]:
def keep_trying(function_loop = get_tables, max_run=10):
    counter = 0
    found_it = False
    while found_it == False and counter < max_run:
        try:
            function_loop()
            found_it = True
        except:
            time.sleep(2)
        counter = counter + 1

In [8]:
rate_site = init_browser(site_url)

In [132]:
dest_txt[0].send_keys(dest_in_txt)
time.sleep(2)
dest_txt[0].send_keys(Keys.RETURN)

In [144]:
def scrape_actions(orig_in_txt = 'Shanghai', dest_in_txt = 'London'):

    rate_site.refresh()


    rate_site.switch_to_default_content()
    # iframe = rate_site.find_elements_by_tag_name('iframe')
    # rate_site.switch_to_frame(iframe[0])

    time.sleep(1)
    orig_txt = rate_site.find_elements_by_id('targetOriginal')
    dest_txt = rate_site.find_elements_by_id('targetDestination')


    orig_txt[0].send_keys(orig_in_txt)
    time.sleep(2)
    orig_txt[0].send_keys(Keys.RETURN)

    dest_txt[0].clear()
    dest_txt[0].send_keys(dest_in_txt)
    time.sleep(3)
    dest_txt[0].send_keys(Keys.RETURN)

    search_btn = rate_site.find_element_by_id('main_a_search')
    search_btn.click()

    time.sleep(3)
    rate_tables = get_tables(rate_site)

    return rate_tables

In [145]:
scrape_actions()

,0,1,2,3,4,5,0,1,2,3,4,5
0,19 days,COSCO SHIPPING Lines,20 Jun (Wed) 11:00,Shanghai,London,EPNW,Direct *,,,23 Jun (Sat),11 Jul (Wed),ITAL UNICA
1,22 days,COSCO SHIPPING Lines,21 Jun (Thu) 00:00,Shanghai,London,AAC,Direct *,,,24 Jun (Sun),15 Jul (Sun),COSCO EUROPE
2,25 days,COSCO SHIPPING Lines,21 Jun (Thu) 23:00,Shanghai,London,CEN,Direct *,,,24 Jun (Sun),18 Jul (Wed),CSCL BOHAI SEA
3,20 days,COSCO SHIPPING Lines,21 Jun (Thu) 23:00,Shanghai,London,CEN,Direct *,,,24 Jun (Sun),13 Jul (Fri),CSCL BOHAI SEA
4,30 days,COSCO SHIPPING Lines,21 Jun (Thu) 00:00,Shanghai,London,AAC,Direct *,,,24 Jun (Sun),23 Jul (Mon),COSCO EUROPE
5,34 days,COSCO SHIPPING Lines,22 Jun (Fri) 21:00,Shanghai,London,AWE1,Direct *,,,25 Jun (Mon),29 Jul (Sun),EVER LUCID
6,22 days,COSCO SHIPPING Lines,22 Jun (Fri) 08:00,Shanghai,London,AAC2,Direct *,,,25 Jun (Mon),16 Jul (Mon),EVER LIBRA
7,21 days,COSCO SHIPPING Lines,22 Jun (Fri) 08:00,Shanghai,London,MPNW,Direct *,,,25 Jun (Mon),15 Jul (Sun),APL YANGSHAN
8,21 days,COSCO SHIPPING Lines,23 Jun (Sat) 14:00,Shanghai,London,AAC4,Direct *,,,26 Jun (Tue),16 Jul (Mon),OOCL SOUTHAMPTON
9,27 days,COSCO SHIPPING Lines,26 Jun (Tue) 08:00,Shanghai,London,AWE4,Direct *,,,28 Jun (Thu),25 Jul (Wed),OOCL CHONGQING
